Los paquetes a usar. Vienen con bibliotecas simples para descargar objetos y bases de datos útiles

In [ ]:
import nltk

# import all the resources for Natural Language Processing with Python
nltk.download("book")
!pip install unidecode
# Regex and standarization
import re
import unidecode
# extra Utils
import random
# Dataframes
!pip3 install pickle5
import pickle5 as pickle
import pandas as pd
# Visualización
import matplotlib.pyplot as plt
import matplotlib
from wordcloud import WordCloud
# Natural Language tk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus.reader import CategorizedPlaintextCorpusReader
from nltk import FreqDist, NaiveBayesClassifier, classify
# subir archivos
from google.colab import files
matplotlib.style.use('ggplot')
pd.options.mode.chained_assignment = None

#Aquí copiamos las bases de datos a nuestro directorio en la nube
! git clone https://github.com/LorenzoLeon/bourbaki_NB_analisis_texto.git

Primero, abramos una base de datos. Hoy vamos a utilizar una lista de tweets mexicanos que usaron el hashtag #covid (o derivados). Estos tweets fueron recopilados directamente desde Twitter.

In [ ]:
tweets_covid = pd.read_pickle('bourbaki_NB_analisis_texto/01_datos/tweets_small.pickle')
# uploaded = files.upload()
# tweets_covid = pd.read_pickle('tweets_small.pickle'

In [ ]:
# Un ejemplo del texto dentro del tweet número 62527
print("text: " + tweets_covid["text"][62527])
print("user_name: " + tweets_covid["user_name"][62527])
print("fecha: " + tweets_covid["fecha"][62527])
#print("id_str: " + tweets_covid["id_str"][62527])

text: Están muy activas las granjas de bots de los derrotados neoliberales

Quieren embaucar incautos con su etiqueta "Dr. Muerte"

Intentan hacernos creer que el semáforo Covid falló

El semáforo no es vacuna

Es un indicador

La única opción viable es: "Sana Distancia"

...no hay más
user_name: Anacruz50923807
fecha: 2020-12-12T21:52:10Z


In [ ]:
# Recordamos nuestra función para quitar emojis? la volveremos a usar aquí
def deEmojify(text):
    patron_regex = re.compile(pattern = "["
                                          u"\U0001F600-\U0001F64F"  # emoticons
                                          u"\U0001F300-\U0001F5FF"  # símbolos & pictogramas
                                          u"\U0001F680-\U0001F6FF"  # transporte & símbolos mapas
                                          u"\U0001F1E0-\U0001F1FF"  # banderas0 (iOS)
                                          u"\U00002500-\U00002BEF"  # chinese char
                                          u"\U00002702-\U000027B0"
                                          u"\U00002702-\U000027B0"
                                          u"\U000024C2-\U0001F251"
                                          u"\U0001f926-\U0001f937"
                                          u"\U00010000-\U0010ffff"
                                          u"\u2640-\u2642"
                                          u"\u2600-\u2B55"
                                          u"\u200d"
                                          u"\u23cf"
                                          u"\u23e9"
                                          u"\u231a"
                                          u"\ufe0f"  # dingbats
                                          u"\u3030"
                                          "]+", flags = re.UNICODE)
    return patron_regex.sub(r'emoji ',text)

In [ ]:
# Copiemos nuestra base y apliquemos los filtros y reemplazos que acabamos de crear
tweets_covid_limpia = tweets_covid.copy()
tweets_covid_limpia['text'] = tweets_covid_limpia['text'].apply(lambda x: deEmojify(x))
tweets_covid_limpia['text'] = tweets_covid_limpia['text'].apply(lambda x: x.lower())
tweets_covid_limpia['text'] = tweets_covid_limpia['text'].apply(lambda x: re.sub(r'https?:\/\/\S*' , ''        , x))
tweets_covid_limpia['text'] = tweets_covid_limpia['text'].apply(lambda x: re.sub(r'@\S*'           , 'mencionTwitter', x))
tweets_covid_limpia['text'] = tweets_covid_limpia['text'].apply(lambda x: re.sub(r"[\W_\d]"        , ' '       , x))
tweets_covid_limpia['text'] = tweets_covid_limpia['text'].apply(lambda x: re.sub(r'\s+'            , ' '       , x))
#quitar caracteres acentuados y 'ñ's
tweets_covid_limpia['text'] = tweets_covid_limpia['text'].apply(lambda x: unidecode.unidecode(x))

También recuperemos nuestras palabras stop, que nos permitirán deshacernos de palabras inservibles en nuestros tweets. 

In [ ]:
stop_words_sp = set(stopwords.words('spanish'))
stop_words_sp = stop_words_sp.union(["mencionTwitter", "covid"])

print(stop_words_sp)

{'tendríamos', 'tuya', 'habremos', 'soy', 'hubieran', 'tuvo', 'tenido', 'habida', 'del', 'suyo', 'seas', 'quienes', 'tuviste', 'seríamos', 'será', 'esto', 'erais', 'sintiendo', 'fuisteis', 'fuera', 'somos', 'nuestro', 'están', 'estarías', 'tuyos', 'el', 'tuvieses', 'tuvieras', 'hube', 'a', 'habíamos', 'sentidos', 'estuvimos', 'antes', 'estuviste', 'en', 'nada', 'estarán', 'hubiste', 'haya', 'sí', 'hubieron', 'habíais', 'hubimos', 'serían', 'estabais', 'han', 'habrías', 'estarás', 'estoy', 'estad', 'sentida', 'estuviesen', 'donde', 'tendrán', 'has', 'que', 'estos', 'teníamos', 'hubiera', 'ya', 'estuvieras', 'habidas', 'la', 'uno', 'tendrá', 'habrían', 'mucho', 'tuviera', 'mía', 'todos', 'muchos', 'nosotras', 'estaríais', 'estaréis', 'como', 'tengamos', 'son', 'esas', 'seáis', 'tuvieseis', 'tuvisteis', 'hubo', 'habrá', 'habréis', 'habríais', 'tuvierais', 'serías', 'hemos', 'serás', 'les', 'tenemos', 'habré', 'tendré', 'estuvieseis', 'y', 'desde', 'está', 'tuvieran', 'para', 'tendrías', '

Creémos nuestro diccionario de las palabras que aparecen en nuestros tweets.

In [ ]:
todas_las_palabras = []
for fila in tweets_covid_limpia["text"]:
    # for w in word_tokenize(unidecode.unidecode(fila), language="spanish"): # quitando acentos
    for w in word_tokenize(fila, language="spanish"):  # sin quitar acentos
        if w not in stop_words_sp:
            todas_las_palabras.append(w)
# nltk nos permite transformar una lista de palabras repetidas en una lista de frequencias de palabras.
todas_las_palabras = FreqDist(todas_las_palabras)
# ahora visualizamos las 15 más comunes y sus frecuencias
print(todas_las_palabras.most_common(15))
# Así, podemos guardar las 5000 palabras más utilizadas en nuestro corpus de tweets
palabras_caracteristicas = list(todas_las_palabras.keys())[:5000]

[('mexico', 3794), ('mas', 3680), ('emoji', 3625), ('salud', 2683), ('distancia', 2582), ('sana', 2419), ('cubrebocas', 2382), ('pandemia', 2210), ('si', 2149), ('mil', 2143), ('gracias', 1906), ('hoy', 1900), ('gobernador', 1869), ('muertes', 1789), ('vacuna', 1762)]


Ahora vamos a organizar nuestros tweets en una lista con categorías, donde por un lado tengamos una lista de palabras dentro del tweet, y del otro, el sentimiento del tweet.

In [ ]:
corpus = [(list(word_tokenize(text, language="spanish")), sentiment)
                 for sentiment, text in zip(tweets_covid_limpia["type"], tweets_covid_limpia["text"])]
# Así podemos además revolverlo para garantizar que no preferimos ningún tipo de tweet
random.shuffle(corpus)

Para poder procesar nuestro tweet, necesitamos que, en vez de las palabras, nos diga cuáles de las palabras de nuestro diccionario contiene. La siguiente función transforma las palabras en columnas de palabras caracteristicas o "features".

In [ ]:
def buscar_caracteristicas(document):
    palabras = set(document)
    caracteristicas = {}
    for w in palabras_caracteristicas:
        caracteristicas[w] = (w in palabras)
    return caracteristicas

In [ ]:
text, sentiment = corpus[1]
carac = buscar_caracteristicas(text)
print(carac)

{'envivo': False, 'inicia': False, 'cruceinformativo': False, 'jefa': False, 'gobierno': False, 'da': False, 'positivo': False, 'declaro': False, 'tres': False, 'dias': False, 'luto': False, 'nacional': False, 'memoria': False, 'fallecidos': False, 'pandemia': False, 'derechoalainformacion': False, 'instruccion': False, 'presidencial': False, 'volver': False, 'rojo': False, 'primero': False, 'explotar': False, 'anaranjados': False, 'posibles': False, 'llegar': False, 'mil': True, 'muertes': False, 'covidemoji': False, 'ocurra': False, 'morenalapandemiademexico': False, 'gobernador': False, 'primer': False, 'donador': False, 'plasma': False, 'hidalgo': False, 'video': False, 'deberia': False, 'ser': False, 'retuiteado': False, 'miles': False, 'veces': False, 'pesadillas': False, 'ojala': False, 'mexico': False, 'olvide': False, 'nunca': False, 'centavos': False, 'individuo': False, 'hizo': False, 'crisis': False, 'deseo': False, 'mal': False, 'solo': False, 'pienso': False, 'si': False,

Ahora, sí. Procesamos todos los tweets en un set de características basado en el diccionario que creamos. Este es la base que utilizaremos para entrenar y modificar nuestro modelo.

In [ ]:
set_caracteristicas = [(buscar_caracteristicas(text), sentiment) for (text, sentiment) in corpus]

In [ ]:
número_tweets = len(set_caracteristicas)
# tomamos el número de Tweets y lo repartimos en 80% (en un entero)
porcentaje_80 = int(round(len(set_caracteristicas)*.8, 0))
# El set de entrenamiento, toma el primer 80% de la base
set_entrenamiento = set_caracteristicas[:porcentaje_80]
# El set para revisar el modelo toma el porcentaje restante
set_prueba = set_caracteristicas[porcentaje_80:]

En primer lugar, utilizaremos un modelo prehecho por nltk para clasificar nuestros tweets. Esto es tan fácil como entrenar el modelo.

In [ ]:
clasificador_NB = NaiveBayesClassifier.train(set_entrenamiento)

Ya entrenado, revisamos su precision con nuestro set de prueba

In [ ]:
print("Precision porcentual del clasificador:", str(round((classify.accuracy(clasificador_NB, set_prueba)) * 100, 2) )+ "%")

Precision porcentual del clasificador: 90.93%


Dependiendo del orden de los tweets (dada la aleatorización) la precisión irá desde 60-80%. 
Nos ayuda más saber acerca de los valores predichos y las probabilides internas de nuestro modelo. Afortunadamente, nltk nos permite ver las características más informativas dentro de nuestro corpus.

In [ ]:
clasificador_NB.show_most_informative_features(15)

Most Informative Features
                criminal = True           negati : positi =     95.9 : 1.0
                  carcel = True           negati : positi =     77.0 : 1.0
                 liquido = True           negati : positi =     72.3 : 1.0
                 fracaso = True           negati : positi =     68.3 : 1.0
                 condesa = True           negati : positi =     62.2 : 1.0
              distribuye = True           positi : negati =     61.8 : 1.0
       medidassanitarias = True           positi : negati =     60.4 : 1.0
                   paseo = True           positi : negati =     57.1 : 1.0
                  olvide = True           negati : positi =     54.8 : 1.0
                      fe = True           positi : negati =     51.8 : 1.0
                     tio = True           negati : positi =     51.1 : 1.0
                  gastar = True           positi : negati =     49.2 : 1.0
                obesidad = True           negati : positi =     44.1 : 1.0

In [ ]:
corpus_brutas = [(list(word_tokenize(text, language="spanish")), sentiment)
                 for sentiment, text in zip(tweets_covid["type"], tweets_covid["text"])]
random.shuffle(corpus_brutas)

todas_las_palabras_brutas = []
for text,sentiment in corpus_brutas:
    for w in text:  
        if w not in stop_words_sp:
            todas_las_palabras_brutas.append(w)
todas_las_palabras_brutas = FreqDist(todas_las_palabras_brutas)

print(todas_las_palabras_brutas.most_common(15))
palabras_caracteristicas_brutas = list(todas_las_palabras_brutas.keys())[:5000]

def buscar_caracteristicas_brutas(document):
    palabras = set(document)
    caracteristica = {}
    for palabra in palabras_caracteristicas_brutas:
        caracteristica[palabra] = (palabra in palabras)
    return caracteristica

set_caracteristicas_brutas = [(buscar_caracteristicas_brutas(text), sentiment) for (text, sentiment) in corpus_brutas]

set_entrenamiento_brutas = set_caracteristicas_brutas[:porcentaje_80]
set_prueba_brutas = set_caracteristicas_brutas[porcentaje_80:]
clasificador_NB_brutas = NaiveBayesClassifier.train(set_entrenamiento_brutas)
print("Precision porcentual del clasificador de raíces:", (classify.accuracy(clasificador_NB_brutas, set_prueba_brutas)) * 100)
clasificador_NB_brutas.show_most_informative_features(15)

[('@', 38977), (',', 29495), ('#', 28428), ('.', 26639), (':', 17036), ('https', 13359), ('?', 11517), ('HLGatell', 9637), ('!', 7991), ('COVID19', 5015), ('El', 4182), ('COVID', 3178), ('lopezobrador_', 3135), ('México', 3029), ('distancia', 2425)]
Precision porcentual del clasificador de raíces: 90.56818181818181
Most Informative Features
                    Hijo = True           positi : negati =    172.8 : 1.0
              aplaudimos = True           positi : negati =    143.7 : 1.0
                 Detente = True           positi : negati =    142.4 : 1.0
             felicidades = True           positi : negati =    118.0 : 1.0
                   Santo = True           positi : negati =    105.7 : 1.0
                 ¡Juntos = True           positi : negati =    105.4 : 1.0
                 Utiliza = True           positi : negati =     98.1 : 1.0
                      Él = True           negati : positi =     97.9 : 1.0
                Refuerza = True           positi : negati

Pero, podemos mejorarlo? podríamos intentar remover todos los acentos, por ejemplo, con unidecode...

In [ ]:
print(tweets_covid_limpia['text'][678023])
print(unidecode.unidecode(tweets_covid_limpia['text'][678023]))

mis respetos para mencionTwitter ayer día cerró en covid a tan sólo de hoy ha rebasado a reino unido en covid para ocupar el lugar mundial como resultado del trabajo sic de mencionTwitter con por de morenavirus mencionTwitter 
mis respetos para mencionTwitter ayer dia cerro en covid a tan solo de hoy ha rebasado a reino unido en covid para ocupar el lugar mundial como resultado del trabajo sic de mencionTwitter con por de morenavirus mencionTwitter 


O tal vez, nos ayude más lematizar el texto que significa encontrar las raices de las palabras. Esto ayuda mucho en lenguajes como el español, donde tenemos decensiones muy complejas. Ej:
Comer -> Comía -> comíamos -> comida -> comidas

In [ ]:
stemmer = SnowballStemmer('spanish')
for w in word_tokenize(texto):
  if w not in stop_words_sp:
    print("Original : " +w.ljust(12)+ " | Raiz: "+stemmer.stem(w))

NameError: ignored

Repitamos todo nuestro clasificador, pero usando las raices de las palabras

In [ ]:
todas_las_palabras_raiz = []
for palabras, sentiment in corpus:
  for w in palabras:
    if w not in stop_words_sp:
      todas_las_palabras_raiz.append(stemmer.stem(w))
todas_las_palabras_raiz = FreqDist(todas_las_palabras_raiz)

print(todas_las_palabras_raiz.most_common(15))
palabras_caracteristicas_raiz = list(todas_las_palabras_raiz.keys())[:5000]

def buscar_caracteristicas_raiz(document):
    palabras = set(document)
    caracteristica = {}
    for palabra in palabras_caracteristicas_raiz:
        caracteristica[palabra] = (palabra in palabras)
    return caracteristica


# print((find_features_stem(corpus.words('positivo_991.txt'))))
set_caracteristicas_raiz = [(buscar_caracteristicas_raiz(text), sentiment) for (text, sentiment) in corpus]

set_entrenamiento_raiz = set_caracteristicas_raiz[:porcentaje_80]
set_prueba_raiz = set_caracteristicas_raiz[porcentaje_80:]
clasificador_NB_raiz = NaiveBayesClassifier.train(set_entrenamiento_raiz)
print("Precision porcentual del clasificador de raíces:", (classify.accuracy(clasificador_NB_raiz, set_prueba_raiz)) * 100)
clasificador_NB_raiz.show_most_informative_features(15)

[('vacun', 4027), ('mexic', 3763), ('muert', 3718), ('covid-', 3714), ('emoji', 2900), ('salud', 2815), ('contagi', 2722), ('san', 2639), ('distanci', 2597), ('mil', 2468), ('cubreboc', 2454), ('gobern', 2238), ('pandemi', 2216), ('cas', 1994), ('hoy', 1905)]
Precision porcentual del clasificador: 76.11363636363636
Most Informative Features
                criminal = True           negati : positi =     92.5 : 1.0
                   among = True           positi : negati =     90.8 : 1.0
                    xoax = True           positi : negati =     87.4 : 1.0
                    only = True           positi : negati =     54.5 : 1.0
                patients = True           positi : negati =     54.5 : 1.0
                    died = True           positi : negati =     54.5 : 1.0
                      fe = True           positi : negati =     53.0 : 1.0
                     oye = True           negati : positi =     39.2 : 1.0
                    peor = True           negati : positi

¿Qué tal que queremos modificar los parámetros internos al modelo? Para eso están todos los algorítmos de scikitlearn. Los importamos rápidamente y verán lo fácil que es utilizar otros modelos.

In [ ]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB

In [ ]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(set_entrenamiento)
print("Precision porcentual del clasificador Multinomial:",nltk.classify.accuracy(MNB_classifier, set_prueba)* 100)

BNB_classifier = SklearnClassifier(BernoulliNB())
BNB_classifier.train(set_entrenamiento)
print("Precision porcentual del clasificador de Bernoulli:",nltk.classify.accuracy(BNB_classifier, set_prueba)* 100)

Precision porcentual del clasificador Multinomial: 0.8888636363636364
Precision porcentual del clasificador de Bernoulli: 0.8854545454545455


¿Recuerdan el factor de Laplace? Este nos permite ajustar las probabilidades especialmente para aquellas caracteristicas que son inusuales. Aquí podemos modificar el 

In [ ]:
MNB_classifier = SklearnClassifier(MultinomialNB(alpha = 5))
MNB_classifier.train(set_entrenamiento)
print("Precision porcentual del clasificador Multinomial:",nltk.classify.accuracy(MNB_classifier, set_prueba)* 100)

BNB_classifier = SklearnClassifier(BernoulliNB(alpha = 5))
BNB_classifier.train(set_entrenamiento)
print("Precision porcentual del clasificador de Bernoulli:",nltk.classify.accuracy(BNB_classifier, set_prueba)* 100)

Precision porcentual del clasificador Multinomial: 87.59090909090908
Precision porcentual del clasificador de Bernoulli: 87.56818181818183


In [ ]:
MNB_classifier = SklearnClassifier(MultinomialNB(alpha = 5))
MNB_classifier.train(set_entrenamiento_raiz)
print("Precision porcentual del clasificador Multinomial raiz:",nltk.classify.accuracy(MNB_classifier, set_prueba_raiz)* 100)

BNB_classifier = SklearnClassifier(BernoulliNB(alpha = 5))
BNB_classifier.train(set_entrenamiento_raiz)
print("Precision porcentual del clasificador de Bernoulli raiz:",nltk.classify.accuracy(BNB_classifier, set_prueba_raiz)* 100)

Precision porcentual del clasificador Multinomial raiz: 73.86363636363636
Precision porcentual del clasificador de Bernoulli raiz: 75.56818181818183


In [ ]:
n = 7
class_labels = MNB_classifier._clf.classes_
feature_names = MNB_classifier._vectorizer.get_feature_names()
topn_class1 = sorted(zip(MNB_classifier._clf.coef_[0], feature_names))[:n]
topn_class2 = sorted(zip(MNB_classifier._clf.coef_[0], feature_names))[-n:]
for coef, feat in topn_class1:
    print( class_labels[0], coef, feat)
print
for coef, feat in reversed(topn_class2):
    print( class_labels[1], coef, feat)

0 -9.290241531170668 'catastrof
0 -9.290241531170668 'covidtrans
0 -9.290241531170668 *calderon
0 -9.290241531170668 *consult
0 -9.290241531170668 *epn
0 -9.290241531170668 *lozoy
0 -9.290241531170668 -felic
1 -2.088772407505031 de
1 -3.126926727136026 del
1 -3.607342267430891 se
1 -3.705993625209656 covid-
1 -3.7357327592622207 salud
1 -4.127743887765166 emoji
1 -4.232722651175561 hoy
